In [1]:
%env CUDA_VISIBLE_DEVICES=1,7
%env TMPDIR=/raid/users/ryan_cheng/tmp
import os
import glob
import re
import json
import random
import time
import pickle
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

np.random.seed(0)

from utils import *
import utils
from consistency_eval import *
from persona_chat_generation import *

try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass

env: CUDA_VISIBLE_DEVICES=1,7
env: TMPDIR=/raid/users/ryan_cheng/tmp
INFO 04-09 03:06:32 __init__.py:190] Automatically detected platform cuda.


In [2]:
with open(os.path.abspath('../ryan_openai.txt'), 'r') as f:
    utils.client = OpenAI(api_key=f.read().rstrip('\n'))

In [3]:
# choose config file to load into global config dictionary in utils.py
# with open("./config/persona_chat/Llama-3.1-70B-Instruct.json", 'r') as f:
#     config_llama = json.load(f)
with open("./config/persona_chat/gpt-4o-mini.json", 'r') as f:
    config_gpt4_mini = json.load(f)

for key, value in config_gpt4_mini.items():
    config[key] = value

# this modifies the global prompts dictionary in utils.py
with open('config/persona_chat/prompts.json', 'r') as f:
        new_prompts = json.load(f)

for key, value in new_prompts.items():
    prompts[key] = value

In [4]:
config['seed'] = 1

# Persona Chat Generation

In [7]:
persona_chat = pd.read_csv('./data/persona/personality.csv')
print("Length:", len(persona_chat))
persona_chat.head()

Length: 8939


,Unnamed: 0,Persona,chat
0,0,i like to remodel homes. i like to go hunting...,"hi , how are you doing ? i am getting ready to..."
1,1,my mom is my best friend. i have four sisters...,"hi , how are you doing today ?\ni am spending ..."
2,2,i had a gig at local theater last night. i wo...,"we all live in a yellow submarine , a yellow s..."
3,3,i am very athletic. i wear contacts. i have b...,hi ! i work as a gourmet cook .\ni do not like...
4,4,i am primarily a meat eater. i am a guitar pl...,how are you doing today\nwhat do you do for ca...


In [8]:
np.random.seed(0)
random_indices = np.array([np.random.choice(len(persona_chat), size=2, replace=False) for _ in range(10)])
random_indices # pairs of indices of Persona to  use in conversation

array([[7710, 4426],
       [5525, 2032],
       [ 574, 6234],
       [2303,  263],
       [3890,  877],
       [8913, 6509],
       [7400, 2988],
       [6578, 4878],
       [5014, 8746],
       [8254, 5155]])

In [16]:
# Generate conversations using personas from persona chat (unneeded)

# file to write output to
write_file = (
        f"data/persona/exp/"
        f"{config['agent1_model']}_{config['seed']}.json"
    )

index_offset = load_stats_file(write_file)

conversations = []
for p1i, p2i in tqdm(random_indices):
    reset_stats()
    conversations.append(generate_conversation(persona_chat.iloc[p1i]["Persona"], persona_chat.iloc[p2i]["Persona"], pturn=1))
    stats['index'] = index_offset
    stats['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    write_stats(write_file)
    index_offset += 1
    

written!!


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [02:04<00:00, 12.40s/it]


In [ ]:
exp_folder = './data/persona/exp'
# config['filename'] = '/nfs/kun2/users/ryan_cheng/consistency_LLMs/data/persona/exp/gpt-4o-mini_1.json'
if config['filename']:
    run_metrics(config['filename'])
else:
    for filename in glob.glob(f'{exp_folder}/*.json'):
        run_metrics(filename)

# Synthetic Persona Chat

In [22]:
persona_chat = pd.read_csv('./data/synthetic-persona/New-Persona-New-Conversations.csv')
persona_chat.head()

,user 1 personas,user 2 personas,Best Generated Conversation
0,I'm moving to a new city to pursue my culinary...,"I am comfortable with the weather, and enjoy s...","User 1: Hi! How are you?\nUser 2: Good, thanks..."
1,I used my artistic skills to teach children ho...,I like to run and do yoga in my spare time.\nM...,User 1: Hey how are you doing?\nUser 2: Doing ...
2,"I enjoy spending time with my pets, and I love...",I am close to my mother and love to spend time...,"User 1: Hello there, how are you doing today?\..."
3,"My cupcake business keeps me busy, but I also ...",I collect memories and seashells from my trave...,User 1: hi there!\nUser 2: Hey! How's your day...
4,The weather is perfect for cheering!\ni listen...,I had to move across country for my job.\nI go...,User 1: How is your day going?\nUser 2: Great!...


In [29]:
np.random.seed(0)
random_indices = np.array([np.random.choice(len(persona_chat), size=2, replace=False) for _ in range(10)])
random_indices # pairs of indices of Persona to  use in conversation

array([[ 9263,  4795],
       [ 1970,  6026],
       [  523,  2596],
       [ 9094,   194],
       [ 9967,  6999],
       [ 3452,  8207],
       [ 1425,  7914],
       [ 6028,  6878],
       [ 1226, 10490],
       [ 7645,  7320]])

In [23]:
persona_chat.iloc[0]["Best Generated Conversation"]

"User 1: Hi! How are you?\nUser 2: Good, thanks for asking! How about yourself?\nUser 1: I'm doing pretty well.  I'm excited to be moving to a new city soon!\nUser 2: Oh that's awesome! What city are you moving to?\nUser 1: I'm moving to Portland, Oregon.  I'm going to be pursuing my culinary dreams there.\nUser 2: That sounds amazing! I love Portland.  I'm originally from there.\nUser 1: Really? That's so cool!  Do you still live there?\nUser 2: No, I moved away a few years ago.  But I still visit my family there often.\nUser 1: That's great.  Do you have any favorite places to visit in Portland?\nUser 2: Yes, I love going to Powell's Books.  It's the largest independent bookstore in the world.\nUser 1: That sounds amazing!  I've never been to Powell's Books before.  I'm definitely going to have to check it out.\nUser 2: You definitely should!  It's a really cool place.\nUser 1: Thanks for the recommendation!  I'm sure I'll love it.\nUser 2: No problem!  I'm glad I could help.\n"

In [25]:
# Generate conversations using personas from persona chat (unneeded)

# file to write output to
write_file = (
        f"data/syn_persona_chat/exp/"
        f"{config['agent1_model']}_{config['seed']}.json"
    )

index_offset = load_stats_file(write_file)

conversations = []
for p1i, p2i in tqdm(random_indices):
    reset_stats()
    conversations.append(generate_conversation(persona_chat.iloc[p1i]["user 1 personas"], persona_chat.iloc[p2i]["user 2 personas"], pturn=1))
    stats['index'] = index_offset
    stats['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    write_stats(write_file)
    index_offset += 1

 20%|██        | 2/10 [00:22<01:29, 11.18s/it]


KeyboardInterrupt: 

In [26]:
def syn_conversation_to_list(conversation: str) -> list[str]:
    pattern = r"(User 1:|User 2:)(.*?)(?=(User 1:|User 2:|$))"
    matches = re.findall(pattern, conversation, re.DOTALL)

    # Combine consecutive entries with the same speaker
    combined_entries = []
    for speaker, message, _ in matches:
        if combined_entries and combined_entries[-1].startswith(speaker):
            combined_entries[-1] += " " + message.strip()
        else:
            combined_entries.append(f"{speaker.strip()} {message.strip()}")

    return combined_entries

In [27]:
syn_conversation_to_list(syn_persona_chat.iloc[0]["Best Generated Conversation"])

['User 1: Hi! How are you?',
 'User 2: Good, thanks for asking! How about yourself?',
 "User 1: I'm doing pretty well.  I'm excited to be moving to a new city soon!",
 "User 2: Oh that's awesome! What city are you moving to?",
 "User 1: I'm moving to Portland, Oregon.  I'm going to be pursuing my culinary dreams there.",
 "User 2: That sounds amazing! I love Portland.  I'm originally from there.",
 "User 1: Really? That's so cool!  Do you still live there?",
 'User 2: No, I moved away a few years ago.  But I still visit my family there often.',
 "User 1: That's great.  Do you have any favorite places to visit in Portland?",
 "User 2: Yes, I love going to Powell's Books.  It's the largest independent bookstore in the world.",
 "User 1: That sounds amazing!  I've never been to Powell's Books before.  I'm definitely going to have to check it out.",
 "User 2: You definitely should!  It's a really cool place.",
 "User 1: Thanks for the recommendation!  I'm sure I'll love it.",
 "User 2: No 

In [32]:
# Use conversations from dataset instead, taking the index of the conversation to be the first of the pair of random indices
orig_conversations = []
for p1i, p2i in tqdm(random_indices):
    conv = syn_persona_chat.iloc[p1i]
    conv_dict = {
        "task_name": 'Synthetic Persona Chat',
        "P1": conv['user 1 personas'],
        "P2": conv['user 2 personas'],
        "conversation": syn_conversation_to_list(conv["Best Generated Conversation"]),
        "pturn": 1, # beginning person (this dataset always starts with User 1)
        "conversation_only": True
        }
    orig_conversations.append(conv_dict)
    

100%|██████████| 10/10 [00:00<00:00, 10979.85it/s]


In [34]:
for conversation in tqdm(orig_conversations):
    if conversation['conversation_only']:
        if config['verbose']:
            print("BEGIN PROMPT CONSISTENCY")
        eval_prompt_consistency(conversation)
        if config['verbose']:
            print("BEGIN ALL LINE CONSISTENCY")
        eval_all_line_consistency(conversation)
        if config['verbose']:
            print("BEGIN PREVIOUS LINE CONSISTENCY")
        eval_prev_line_consistency(conversation)
        if config['verbose']:
            print("BEGIN SURVEY CONSISTENCY")
        eval_survey_consistency(conversation)

with open('data/persona/syn_persona_chat.json', 'w') as f:
        json.dump(orig_conversations, f, indent=4)

 30%|███       | 3/10 [10:36<24:45, 212.24s/it]


KeyboardInterrupt: 

# Anthology of Backstories

## Download Anthology

In [24]:
%pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [32]:
import gdown

url = "https://drive.google.com/file/d/18ctlWJXfckgAsb0yC-vSi5sTlid4xFQk/view"
output_path = "./data/anthology/Meta-Llama-3-70B_demographics_survey+political_affiliation_batch_1+2+3_no_word_cutoff.pkl.zip"
gdown.download(url, output_path, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=18ctlWJXfckgAsb0yC-vSi5sTlid4xFQk
From (redirected): https://drive.google.com/uc?id=18ctlWJXfckgAsb0yC-vSi5sTlid4xFQk&confirm=t&uuid=b9628dad-50ae-4340-99e0-812f195bbe31
To: /nfs/kun2/users/ryan_cheng/consistency_LLMs/data/anthology/Meta-Llama-3-70B_demographics_survey+political_affiliation_batch_1+2+3_no_word_cutoff.pkl.zip
100%|██████████| 173M/173M [00:01<00:00, 138MB/s]  


'./data/anthology/Meta-Llama-3-70B_demographics_survey+political_affiliation_batch_1+2+3_no_word_cutoff.pkl.zip'

In [33]:
!unzip ./data/anthology/Meta-Llama-3-70B_demographics_survey+political_affiliation_batch_1+2+3_no_word_cutoff.pkl.zip -d ./data/anthology/
!rm ./data/anthology/Meta-Llama-3-70B_demographics_survey+political_affiliation_batch_1+2+3_no_word_cutoff.pkl.zip

Archive:  ./data/anthology/Meta-Llama-3-70B_demographics_survey+political_affiliation_batch_1+2+3_no_word_cutoff.pkl.zip
  inflating: ./data/anthology/Meta-Llama-3-70B_demographics_survey+political_affiliation_batch_1+2+3_no_word_cutoff.pkl  


## Load Anthology Profiles

In [35]:
file_path = "data/anthology/Meta-Llama-3-70B_demographics_survey+political_affiliation_batch_1+2+3_no_word_cutoff.pkl"

try:
    with open(file_path, "rb") as f:
        df = pd.read_pickle(file_path)
except (EOFError, pickle.UnpicklingError) as e:
    print("Error loading pickle file:", e)

In [36]:
def count_sentences(paragraph):
    # Split sentences using punctuation (., !, ?) as well as commas
    sentences = re.split(r'[.!?]', paragraph)  # Split by sentence-ending punctuation
    return len(sentences)

In [37]:
personas = []

# The target phrase
target_phrase_start = "Answer:"
target_phrase_end = "Question:"

for text in df["age_category_4_llm_parsing_prompt"]:
    # Find the start index of the target phrase and extract text after it
    start_index = text.find(target_phrase_start) 
    stop_index = text.find(target_phrase_end, start_index) 
    persona = text[start_index+len(target_phrase_start)+1:stop_index]
    count = count_sentences(persona)
    if count > 20:
        personas.append({"persona": persona, "len": count})

len(personas)

4895

In [38]:
print(personas[0]['persona'])

I am in my late 30s, grew up in a small town in the mountains of southern Appalachia, left home at 18, was in the army, and worked for some time in the auto industry before starting my current job. What matters to me are my wife and children, faith, family, freedom, and the sanctity of life.

I didn’t have any exposure to science growing up. I grew up with the Bible and the world of make-believe. The first thing I remember being excited about was “The Little Engine That Could” which was about having a can-do attitude. Then I became interested in dinosaurs and started reading a lot about them. I read books by Sir David Attenborough and Carl Sagan. Then I got into Star Trek. That’s what I’m still most excited about. I have no idea why I’m not excited about sci-fi movies, and maybe I’m missing out on something, but I just don’t have the same enthusiasm for movies that I do for the science I read in books.

I’m interested in biology because it is so fundamental to all other sciences. I don

## Generate Details about Personas

Only has to be run once, if ./data/anthology/personas_updated.json is missing or if new persona backgrounds are to be sampled and extended

In [39]:
np.random.seed(0)
random_indices_anthology = np.random.choice(len(personas), size=200, replace=False).astype(int)
random_indices_anthology # random indices to generate profiles for

array([4029, 1755, 3834, 2078, 4129, 3819, 3722, 1974,  495, 3147, 3803,
       3837,  965, 1011,  142, 3371, 2955, 3127, 2744, 4813, 4158, 3699,
        311, 2814, 4128, 1393, 4841, 3566,  716,  891, 1877, 1934,   49,
        599,  556, 1428, 1979, 2443, 3568, 2574, 3756,  298,  861, 3011,
        651, 4509, 3073,  202, 3710,  148, 3170, 1489, 4761, 3460, 1272,
       4044,  898, 4548, 1687,  154,  158, 1800, 1515, 4475, 3071, 4073,
         39, 3615,  624, 1936, 2399, 4828, 3712, 2165, 2601, 4070, 2661,
       4371, 4555, 2884,  348, 4146, 1691,  873,  402, 4607, 3420,  130,
       4574, 4339, 2396, 3133, 3309,  541, 2724,  308, 2779,  933, 3582,
        562, 2519,  500, 3389, 1108,  534, 1692, 4481, 4883, 2471, 3608,
       1675, 1738, 4343, 4716, 3060, 3786,  113,  489,   98, 2290,  636,
       3570, 3328,  499, 4837, 2527, 3900,   42, 1437, 1051, 3975, 3320,
       1172, 4052, 4266, 4430, 2682, 2381, 1450, 4376, 3065,  394, 1900,
        125, 1997, 4061, 1794, 1419, 2509, 1326, 34

In [40]:
persona_prompt = "Provide a biography that includes all of the following attributes. It should not be presented as a list, but as if someone was providing the information naturally in an introduction in the first person. It should be contained to one paragraph. Here is a biography that you can add details to contain all of the attributes below.\n"

attributes_prompt = """Here are the attributions that must be in the biography:
- Name
- Age
- Where they're from
- Where their parents are from
- Religion and extent it is adhered to
- Socioeconomic status - current
- Socioeconomic status - childhood
- Siblings (including name, age, relationship)
- Languages and dialects spoken
- Sexual orientation
- Gender identity
- Relationship status
- Significant past relationships
- Occupation (current and past)
- Education
- Cultural influences
- Political views
- Health and wellness
- Hobbies and interests
- Values and beliefs outside religion
- Fears and anxieties
- Life goals and ambitions
- Defining life experiences
- Friendship circles
- Daily routine and habits
- Pet ownership (current and past)
- Favorite media
- Living situation
- Places traveled to"""


In [83]:
personas_updated = []
personas_small = np.array(personas)[random_indices_anthology]
for persona in tqdm(personas_small):
    agent_prompt = str(persona_prompt) + "Biography:" + str(persona) + "\n" + str(attributes_prompt)
    agent_prompt+= "\nUpdated Biography:"
    agent_answer = completion_create(
            config_gpt4_mini["agent1_model"],
            config_gpt4_mini,
            agent_prompt, 
            config_gpt4_mini["agent1_model"])
    personas_updated.append(agent_answer)


100%|██████████| 100/100 [03:53<00:00,  2.34s/it]


In [90]:
with open('data/anthology/personas_updated.json', 'w') as f:
        json.dump([{"persona": personas_updated[i], "index": int(random_indices_anthology[i])} for i in range(len(personas_updated))], f, indent=4)

## Load Detailed Anthology Personas and Generate Conversations

In [43]:
with open('data/anthology/personas_updated.json', 'r') as f:
        personas_updated = json.load(f)

In [44]:
np.array(personas_updated).reshape(-1, 2)[0]

array([{'persona': 'My name is Joon, I’m 22 years old, and I was born in South Korea but immigrated to the US at the age of four. Growing up in a Korean household, I learned the values of hard work and family from my grandparents, who faced significant hardships in their lives, leading to my current belief that no one deserves to be poor and that conversation is vital for progress. Now a college graduate from the University of Washington studying law and business, I’m passionate about social justice and engage with diverse opinions to challenge hypocrisy within society, all while navigating my identities as a bisexual, non-binary individual in a rapidly changing world.', 'index': 4029},
       {'persona': 'Hello, I’m Dr. Kenji Takahashi, a 60-year-old Asian American male from San Francisco, where my parents, both physicians from Japan, instilled in me the importance of community service and free-thinking. Growing up as an only child in a liberal household, I was immersed in both Englis

In [46]:
# Generate conversations using personas from persona chat (unneeded)

# file to write output to
write_file = (
        f"data/anthology/exp/"
        f"{config['agent1_model']}_{config['seed']}.json"
    )

index_offset = load_stats_file(write_file)

conversations = []
for p_dict1, p_dict2 in tqdm(np.array(personas_updated[:20]).reshape(-1, 2)):
    reset_stats()
    conversations.append(generate_conversation(p_dict1["persona"],  p_dict2["persona"], pturn=1))
    stats['index'] = index_offset
    stats['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    write_stats(write_file)
    index_offset += 1

written!!


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [02:11<00:00, 13.13s/it]


In [48]:
exp_folder = './data/anthology/exp'
# config['filename'] = '/nfs/kun2/users/ryan_cheng/consistency_LLMs/data/persona/exp/gpt-4o-mini_1.json'
config['filename'] = '/nfs/kun2/users/ryan_cheng/consistency_LLMs/data/anthology/exp/gpt-4o-mini_1.json'
if config['filename']:
    run_metrics(config['filename'])
else:
    for filename in glob.glob(f'{exp_folder}/*.json'):
        run_metrics(filename)

Begin metrics: /nfs/kun2/users/ryan_cheng/consistency_LLMs/data/anthology/exp/gpt-4o-mini_1.json




100%|██████████| 10/10 [15:45<00:00, 94.54s/it]

End metrics: /nfs/kun2/users/ryan_cheng/consistency_LLMs/data/anthology/exp/gpt-4o-mini_1.json




## Summary prompt/answers

In [59]:
print(personas_updated[0])

My name is James, and I’m in my late 30s, hailing from a small town in the mountains of southern Appalachia, where my parents, originally from the deep South, instilled strong values in me, centered around faith and family. I grew up in a modest household, the oldest of three siblings, with my sister Sarah, who is 34, and my brother Mike, who is 28; our upbringing was rich in stories and imagination, largely influenced by the Bible, but it wasn't until I discovered my passion for biology and the wonders of the universe through the words of great thinkers like Sir David Attenborough that I truly began to understand the world around me. Currently, I work in the tech industry after a diverse career path that included time in the army and the auto industry, while my wife and our two children remain my top priorities, and together we navigate our daily life filled with curiosity, exploration, and values that stress the importance of freedom and the sanctity of life.


In [45]:
summary_prompt = """
    From the following biography, generate one sentence summaries for each of the attributes with similar structure to the accompanying examples:
        name = ""  # "My name is Jane Doe."
        age = ""  # "I am 35 years old."
        location_from = ""  # "I am from Houston, Texas."
        location_mother_from = "" # "My mother is from Capetown, South Africa."
        location_father_from = "" # "My father is from Mumbai, India."
        religion = "" # "I am Christian.", "I was raised Jewish but don't practice."
        socioeconomic_status = "" # I am middle class.", "I am a wealthy homeowner.", "I am struggling to make ends meet."
        siblings = "" # "I am an only child." "I have a brother and a sister."
        languages_spoken = "" # "I speak only English.", "I speak Portuguese, English, and Bengali."
        sexual_orientation = "" # "I am heterosexual.", "I'm bisexual."
        gender_identity = "" # "I am a woman.", "I identify as nonbinary."
        relationship_status = "" # "I'm currently single.", "I have been married for 4 years."
        significant_past_relationships = "" # "I was engaged for 2 years but it was called off.", "I have never been in a long term relationship."
        occupation_current = "" # "I am a landscaper.", "I do small business consulting."
        occupation_past = "" # "I used to be a frycook in high school.", "In college I worked as a librarian's assistant."
        education = "" # "I have my G.E.D.", "I have a bachelors in Finance from Northeastern University."
        cultural_influences = "" # "I grew up in a devout Mormon community.", "I grew up in a Nigerian Igbo community."
        political_views = "" # "I don't follow politics.", "I am a democratic socialist."
        health = "" # "I am very healthy and exercise often, but have a bad knee.", "I am legally blind."
        hobbies_and_interests = "" # "I enjoy painting and hiking.", "I am a huge history buff."
        beliefs_and_values = "" # "I believe minimizing our carbon footprint is imperative.", "I am vegan for moral reasons."
        fears_and_anxieties = "" # "I am deathly afraid of heights.", "I have anxiety with personal conflict."
        life_goals_and_ambitions = "" # "I hope to retire at an early age.", "I want to start my own company.", "All I want out of life is a big family."
        defining_life_experiences = "" # "I was orphaned at a young age.", "I would bake with my grandma every Sunday growing up."
        friendship_circles = "" # "I've had the same few best friends for years.", "I have no close friends but run in a lot of social circles."
        daily_routine_and_habits = "" # "I do a morning yoga routine every morning before work.", "I spend an hour reading every night before I go to bed."
        pet_ownership = "" # "I don't have any pets.", "I have a black laborador retriever named Sparky."
        favorite_media = "" # "I have rewatched the TV show Friends a dozen times.", "I watch football games every night."
        living_situation = "" # "I currently live by myself in a studio apartment.", "I live in a single-family home with my husband and two children."
        places_traveled = "" # "I have never left my home city.", "I have traveled to most of South America, and a few countries in Europe."

    """


In [54]:
summary_prompt += "\nBiography:" + personas_updated[0] + "\nFormat the output as above, with the attribute from the class matching the key for the attribute:\n name: <summary>\nage: <summary\n..."

summary_answer = completion_create(
        config_gpt4_mini["agent1_model"],
        config_gpt4_mini,
        summary_prompt,
        config_gpt4_mini["agent1_model"])


In [55]:
print(summary_answer)

name = "My name is James."  
age = "I am in my late 30s."  
location_from = "I am from a small town in the mountains of southern Appalachia."  
location_mother_from = "My mother is originally from the deep South."  
location_father_from = "My father is also originally from the deep South."  
religion = "I was raised with strong values centered around faith."  
socioeconomic_status = "I grew up in a modest household."  
siblings = "I have a sister named Sarah, who is 34, and a brother named Mike, who is 28."  
languages_spoken = "I speak English."  
sexual_orientation = ""  # Not specified  
gender_identity = ""  # Not specified  
relationship_status = "I am married."  
significant_past_relationships = ""  # Not specified  
occupation_current = "I work in the tech industry."  
occupation_past = "I have had a diverse career path that included time in the army and the auto industry."  
education = ""  # Not specified  
cultural_influences = "My upbringing was rich in stories and imaginati

# Consistency Evaluation

Moved under each section (codeblocks with run_metrics)

Can either use run_metrics(filename) or explicitly call each eval function (see Synthetic Persona Chat section)